In [1]:
! pip install graphframes



^C


In [ ]:
! pyspark --packages graphframes:graphframes:0.8.2-spark3.0-s_2.12


^C


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("GraphFramesExample") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
    .getOrCreate()


In [3]:

from graphframes import GraphFrame
edges = spark.read.csv("musae_git_edges.csv", header=True, inferSchema=True)
edges = edges.withColumnRenamed("id_1", "src").withColumnRenamed("id_2", "dst")

vertices = spark.read.csv("musae_git_target.csv", header=True, inferSchema=True)
vertices = vertices.withColumnRenamed("id", "id").select("id", "name", "ml_target")


In [4]:
from graphframes import GraphFrame

graph = GraphFrame(vertices, edges)


C:\Users\krish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [5]:
lpa_result = graph.labelPropagation(maxIter=5)
lpa_result.show()


C:\Users\krish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyspark\sql\dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-----+----------------+---------+-----+
|   id|            name|ml_target|label|
+-----+----------------+---------+-----+
|37268|        Chloejay|        0|35773|
|19021|AntoinedeChassey|        0|35773|
|28730|      aakashlpin|        0|35773|
|23776|    eroispaziali|        0| 5547|
|31037|             dok|        0|35773|
|34207|        pixeline|        0|35773|
|29127|   geonwoo-jeong|        0|35773|
| 9831|        ekntrtmz|        0|35773|
| 5354|       cbarcenas|        0|35773|
|32676|          sasha0|        0|35773|
| 4926|   dustinmoorman|        0|35773|
|29270|      advaitsave|        1| 5547|
|14609|      huangjihua|        0|31126|
|21377|    martinyunify|        1|35773|
|11852|  JeroenReumkens|        0|35773|
| 8390|        diegonvs|        0|35773|
|28761|        sosedoff|        0|35773|
|10837|    lch14forever|        1|26963|
| 4992|        eirslett|        0|35773|
|20894|         fakyras|        1|35773|
+-----+----------------+---------+-----+
only showing top

In [6]:
communities=lpa_result

In [7]:
from pyspark.sql.functions import col, count, lit, when

edges_with_communities = edges\
    .join(lpa_result.selectExpr("id as src", "label as src_label"), on="src") \
    .join(lpa_result.selectExpr("id as dst", "label as dst_label"), on="dst")



In [8]:
intra_edges = edges_with_communities.filter(col("src_label") == col("dst_label"))

total_edges = edges.count()
intra_community_edges = intra_edges.count()


In [9]:

modularity = intra_community_edges / total_edges
print(f"Modularity Score: {modularity}")

Modularity Score: 0.7995557139545264
